<a href="https://colab.research.google.com/github/twister0316/Public/blob/main/DART_API_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas opendartreader

In [ ]:
import OpenDartReader
import pandas as pd
import time

# ---------------------------------------------------------
# [설정] 본인의 API 키와 검색할 연도를 입력하세요
# ---------------------------------------------------------
API_KEY = '20640a52d2ad1da701eb6192953ef9e16ebd7148'  # DART 인증키 (40자리)
TARGET_YEAR = '2024'                   # 검색할 연도
REPORT_CODE = '11011'                  # 11011: 사업보고서 (최종 확정)

# 객체 생성
dart = OpenDartReader(API_KEY)

def get_company_revenue(corp_code, year):
    """특정 기업의 재무제표에서 '매출액'만 뽑아오는 함수"""
    try:
        # 재무제표 조회 (연결재무제표 우선, 없으면 개별)
        fs = dart.finstate(corp_code, year, REPORT_CODE)

        if fs is None or fs.empty:
            return None

        # 매출액(Revenue) 계정 찾기 (account_nm이 '매출액' 또는 '수익(매출액)')
        revenue_row = fs[fs['account_nm'].str.contains('매출액|수익', na=False)]

        # 당기 금액 가져오기 (thstrm_amount)
        if not revenue_row.empty:
            # 3자리 콤마 제거 후 숫자로 변환
            revenue = revenue_row.iloc[0]['thstrm_amount']
            return revenue

    except Exception as e:
        print(f"Error fetching {corp_code}: {e}")
        return None

# ---------------------------------------------------------
# [메인 로직] 기업 리스트 가져오기 및 매출액 매칭
# ---------------------------------------------------------
print("1. 전체 기업 목록(고유번호)을 가져옵니다...")
# DART에 등록된 모든 기업 리스트 로딩 (약 10만개 중 상장사만 필터링도 가능)
corp_list = dart.corp_codes

# ★ 테스트용: 너무 오래 걸리니 상위 10개 기업만 먼저 해보세요 ★
target_corps = corp_list.head(10)
# 실제 전체 실행 시 위 줄을 주석 처리하고 아래 줄을 푸세요:
# target_corps = corp_list[corp_list['stock_code'].notnull()] # 상장사(종목코드 있는 곳)만 추출

result_list = []

print(f"2. {len(target_corps)}개 기업의 매출액 조회를 시작합니다.")

for idx, row in target_corps.iterrows():
    c_code = row['corp_code']
    c_name = row['corp_name']

    # 매출액 가져오기 함수 호출
    rev = get_company_revenue(c_code, TARGET_YEAR)

    # 결과 저장
    result_list.append({
        '회사명': c_name,
        '종목코드': row['stock_code'],
        '고유번호': c_code,
        '매출액(원)': rev
    })

    print(f"[{idx+1}/{len(target_corps)}] {c_name} : {rev}")

    # API 호출 제한 방지 (너무 빠르면 차단될 수 있음)
    time.sleep(0.2)

# ---------------------------------------------------------
# [저장] 결과를 엑셀로 저장
# ---------------------------------------------------------
df_final = pd.DataFrame(result_list)
file_name = f"기업매출액리스트_{TARGET_YEAR}.xlsx"
df_final.to_excel(file_name, index=False)

print(f"\n완료! '{file_name}' 파일이 저장되었습니다.")

1. 전체 기업 목록(고유번호)을 가져옵니다...
2. 10개 기업의 매출액 조회를 시작합니다.
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
Error fetching 00434003: '<' not supported between instances of 'str' and 'int'
[1/10] 다코 : None
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
Error fetching 00430964: '<' not supported between instances of 'str' and 'int'
[2/10] 굿앤엘에스 : None
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
Error fetching 00388953: '<' not supported between instances of 'str' and 'int'
[3/10] 크레디피아제이십오차유동화전문회사 : None
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
Error fetching 00179984: '<' not supported between instances of 'str' and 'int'
[4/10] 연방건설산업 : None
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
Error fetching 00420143: '<' not supported between instances of 'str' and 'int'
[5/10] 브룩스피알아이오토메이션잉크 : None
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
Error fetching 00401111: '<' not supported between instances of 'str' and 'int'
[6/10] 매경아이비아이 : None
{'status': '013', 'message': '조회된 데이타가 없습니다

In [ ]:
!pip install opendartreader
import OpenDartReader
import pandas as pd
import time

# ---------------------------------------------------------
# [설정] 본인의 API 키와 검색할 연도를 입력하세요
# ---------------------------------------------------------
API_KEY = '20640a52d2ad1da701eb6192953ef9e16ebd7148'
TARGET_YEAR = '2023'                   # 매출액 기준 연도
REPORT_CODE = '11011'                  # 11011: 사업보고서

# 객체 생성
dart = OpenDartReader(API_KEY)

# ---------------------------------------------------------
# [함수 1] 매출액 가져오기
# ---------------------------------------------------------
def get_revenue(corp_code):
    try:
        # 재무제표 조회 (연결 우선)
        fs = dart.finstate(corp_code, TARGET_YEAR, REPORT_CODE)
        if fs is None or fs.empty:
            return None

        # 매출액(Revenue) 찾기
        #revenue_row = fs[fs['account_nm'].str.contains('매출액|수익', na=False)]
        revenue_row = fs[fs['account_nm'].str.contains('매출액', na=False)]

        if not revenue_row.empty:
            # 금액 문자열을 숫자로 변환
            amt_str = revenue_row.iloc[0]['thstrm_amount']
            return int(amt_str.replace(',', ''))

    except Exception:
        return None
    return None

# ---------------------------------------------------------
# [함수 2] 기업 개황(주소, 홈페이지 등) 가져오기
# ---------------------------------------------------------
def get_overview(corp_code):
    try:
        data = dart.company(corp_code)
        if data:
            return {
                '대표자': data.get('ceo_nm'),
                '주소': data.get('adres'),
                '홈페이지': data.get('hm_url'),
                '전화번호': data.get('phn_no'),
                '설립일': data.get('est_dt'),
                '업종코드': data.get('induty_code')
            }
    except Exception:
        pass
    return {}

# ---------------------------------------------------------
# [메인] 전체 상장사 리스트 로딩 및 크롤링
# ---------------------------------------------------------
print("1. 전체 기업 목록을 불러옵니다...")
corp_list = dart.corp_codes

# ★ 중요: 상장사(종목코드가 있는 기업)만 필터링 ★
# 비상장사까지 다 하면 10만 개가 넘어 하루 안에 못 끝냅니다.
target_corps = corp_list[corp_list['stock_code'].notnull()]
#target_corps = corp_list[corp_list['stock_code'].notnull()].head(1)

print(f"2. 총 {len(target_corps)}개 상장 기업의 정보를 조회합니다. (시간이 좀 걸립니다)")

result_list = []

for idx, row in target_corps.iterrows():
    c_code = row['corp_code']
    #c_code = row['에이스침대']
    c_name = row['corp_name']
    stock_code = row['stock_code']

    # 1) 개황 정보 조회
    overview = get_overview(c_code)

    # 2) 매출액 조회
    revenue = get_revenue(c_code)

    # 3) 데이터 합치기
    item = {
        '회사명': c_name,
        '종목코드': stock_code,
        '매출액(원)': revenue,
        '대표자': overview.get('대표자', ''),
        '주소': overview.get('주소', ''),
        '홈페이지': overview.get('hm_url', ''),
        '전화번호': overview.get('phn_no', ''),
        '설립일': overview.get('est_dt', ''),
        '업종코드': overview.get('induty_code', '')
    }

    result_list.append(item)

    # 진행 상황 출력 (10개마다)
    if idx % 10 == 0:
        print(f"[{idx}/{len(target_corps)}] {c_name} 완료")

    # API 과부하 방지 (중요)
    time.sleep(0.15)

# ---------------------------------------------------------
# [저장]
# ---------------------------------------------------------
df = pd.DataFrame(result_list)
file_name = f"상장기업_전체정보_{TARGET_YEAR}.xlsx"
df.to_excel(file_name, index=False)

print(f"\n모든 작업이 완료되었습니다! '{file_name}' 파일을 확인하세요.")

1. 전체 기업 목록을 불러옵니다...
2. 총 114740개 상장 기업의 정보를 조회합니다. (시간이 좀 걸립니다)
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
[0/114740] 다코 완료
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
[10/114740] 포스미디어 완료
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'status': '013', 'message': '조회된 데이타가 없습니다.'}
{'s

# 새 섹션

In [ ]:
import OpenDartReader
import pandas as pd
import time

# ---------------------------------------------------------
# [설정] 본인의 API 키와 검색할 연도를 입력하세요
# ---------------------------------------------------------
API_KEY = '20640a52d2ad1da701eb6192953ef9e16ebd7148'
TARGET_YEAR = '2022'                   # 매출액 기준 연도
REPORT_CODE = '11014'                  # 11011: 사업보고서

# 객체 생성
dart = OpenDartReader(API_KEY)
corp_code = '우신투자자문'
fs = dart.finstate(corp_code, 2023, reprt_code = '11011')
fs.head()
#fs[['account_nm','frmtrm_amount']]


{'status': '013', 'message': '조회된 데이타가 없습니다.'}



""
